In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

* 必要なライブラリをインポート

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

* 学習データをデータフレームに読み込み


In [ ]:
#データの読み込み
train = pd.read_csv("/kaggle/input/uci-wholesale-customers-data/Wholesale customers data.csv")

* 学習データのレコード数とカラム数を確認

In [ ]:
print(train.shape)

* 学習データのデータ表示(最初の5行目)

In [ ]:
train.head()

* 学習データのデータ定義を確認

In [ ]:
train.info()

* 学習データの数値項目の統計要約量を表示

In [ ]:
train.describe()

* pandas-profilingを使用しデータ概要を一括確認

In [ ]:
# import pandas_profiling as pdp
# pdp.ProfileReport(train)

* 不要なカラムを削除(販売チャネル、各学習の地域)

In [ ]:
# 不要なカラムを削除(販売チャネル、各顧客の地域)
del(train['Channel'])
del(train['Region'])
train.head()

# 【演習】k-means法(非階層型クラスタリング)

* k-meansにてクラスター分析を実行(4分割)

In [ ]:
from sklearn.cluster import KMeans

# k-meansを使用し、4分割する
pred = KMeans(n_clusters=4).fit_predict(train)

# クラスターIDを元データに設定
train['cluster_id'] = pred


* クラスターIDごとにレコード件数を表示

In [ ]:
#クラスターIDごとの件数を表示
train.groupby('cluster_id')['cluster_id'].count()

* クラスターIDごとに変数をヒストグラム表示

In [ ]:
g=sns.FacetGrid(train,col="cluster_id")
g=g.map(sns.distplot,"Fresh")
g.add_legend()

* クラスターIDごとに各変数を集計

In [ ]:
#クラスターIDごとに集計する
train_sum = train.groupby('cluster_id').sum()
train_sum.head()

* 集計した変数を積み上げ棒グラフ表示

In [ ]:
#積み上げ棒グラフをクラスターID別に表示
train_sum.plot.bar(stacked=True)

* クラスターIDごとに各変数を集計(百分率表示)

In [ ]:
train_sum_100p = train_sum.apply(lambda x:x/sum(x),axis=1)
train_sum_100p

* 集計した変数(百分率表示)を積み上げ棒グラフ表示

In [ ]:
#積み上げ棒グラフをクラスターID別に表示
train_sum_100p.plot.bar(stacked=True)

# 【演習】ウォード法(階層型クラスタリング)

* ワード法を使用し階層型クラスタリングを実行

In [ ]:
from scipy.cluster.hierarchy import linkage,dendrogram,fcluster

#ユークリッド距離とウォード法を使用して階層型クラスタリングを行う
Z = linkage(train, method='ward', metric='euclidean')
pd.DataFrame(Z)


* 樹形図(デンドログラム)を作成

In [ ]:
import matplotlib.pyplot as plt
fig2, ax2 = plt.subplots(figsize=(20,5))

# 樹形図を作成
ax2 = dendrogram(Z)
fig2.show()


* クラスタ数に３を指定してクラスタリング

In [ ]:
from scipy.cluster.hierarchy import fcluster

# クラスタ数を指定してクラスタリング
clusters = fcluster(Z, t=3, criterion='maxclust')
for i, c in enumerate(clusters):
    print(i, c)


# 【演習】エルボー法

* エルボー法から最適なクラスター数を算出

In [ ]:
# エルボー方による推定。クラスター数を1から10に増やして、それぞれの距離の総和を求める
dist_list =[]
for i in range(1,10):
    kmeans= KMeans(n_clusters=i, init='random', random_state=0)
    kmeans.fit(train)
    dist_list.append(kmeans.inertia_)
    
# グラフを表示
plt.plot(range(1,10), dist_list,marker='+')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
